In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import utils

import numpy as np
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/data/chatbot/dialogs.txt', 'r') as f:
  lines = [i.strip() for i in f]
# questions = np.array(lines[:-1])
questions = np.array([line.split() for line in lines[:-1]])
# answers = np.array(list(map(lambda x: f'<bos> {x} <eos>', lines[1:])))
answers = np.array([(f'<bos> {line} <eos>').split() for line in lines[1:]])

<ipython-input-5-cea66cb0a902>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  questions = np.array([line.split() for line in lines[:-1]])
<ipython-input-5-cea66cb0a902>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  answers = np.array([(f'<bos> {line} <eos>').split() for line in lines[1:]])


In [ ]:
vocabulary_size = 5000
tokenizer = Tokenizer(num_words=vocabulary_size,  oov_token = 'unk')
tokenizer.fit_on_texts(answers[:1001])
vocabluary_items = list(tokenizer.word_index.items())
vocabulary_size = len(vocabluary_items) + 1

print(vocabluary_items[:100])
print(vocabulary_size)

[('unk', 1), ('<bos>', 2), ('<eos>', 3), (',', 4), ('?', 5), ('.', 6), ('я', 7), ('а', 8), ('и', 9), ('у', 10), ('в', 11), ('не', 12), ('ты', 13), ('люблю', 14), ('на', 15), ('!', 16), ('меня', 17), ('как', 18), ('это', 19), ('что', 20), ('очень', 21), ('с', 22), ('привет', 23), ('да', 24), ('мне', 25), ('есть', 26), ('но', 27), ('тоже', 28), ('тебя', 29), ('чем', 30), ('вы', 31), ('вот', 32), ('вас', 33), ('дела', 34), ('нет', 35), ('по', 36), ('хорошо', 37), ('о', 38), ('любишь', 39), ('зовут', 40), ('так', 41), ('то', 42), ('к', 43), ('время', 44), ('ну', 45), ('животных', 46), ('нравится', 47), ('работаю', 48), ('за', 49), ('живу', 50), ('круто', 51), ('из', 52), ('ещ', 53), ('может', 54), ('работаешь', 55), ('тебе', 56), ('занимаешься', 57), ('или', 58), ('много', 59), ('уже', 60), ('хочу', 61), ('приятно', 62), ('лет', 63), ('сейчас', 64), ('их', 65), ('пока', 66), ('когда', 67), ('кем', 68), ('была', 69), ('себе', 70), ('вообще', 71), ('где', 72), ('здорово', 73), ('учусь', 74),

In [ ]:
list(tokenizer.word_index.items())[:10]

[('unk', 1),
 ('<bos>', 2),
 ('<eos>', 3),
 (',', 4),
 ('?', 5),
 ('.', 6),
 ('я', 7),
 ('а', 8),
 ('и', 9),
 ('у', 10)]

In [ ]:
tokenizer.texts_to_sequences(['<bos> привет как дела ?'])

[[1, 23, 18, 34]]

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256 # скрытая размерность
num_samples = 10000
embedding_dim = 256  # Размерность эмбеддинга
# max_len = 20

In [ ]:
def prepare_data(phrases):
  tokenized = tokenizer.texts_to_sequences(phrases)
  max_len = max([ len(x) for x in tokenized])
  # max_len = 20

  padded = pad_sequences(tokenized, maxlen=max_len, padding='post')
  encoded = np.array(padded)
  return encoded, max_len

In [ ]:
encoder_input, max_len_quest = prepare_data(questions[:1000])

In [ ]:
# phrases = answers[:1001][:-1]
phrases = [line[:-1] for line in answers[:1001]]
tokenized = tokenizer.texts_to_sequences(phrases)
max_len_answ = max([ len(x) for x in tokenized])
decoder_input = pad_sequences(tokenized, maxlen=max_len_answ, padding='post')

In [ ]:
# phrases = answers[:1001][1:]
phrases = [line[1:] for line in answers[:1001]]
tokenized = tokenizer.texts_to_sequences(phrases)
padded = pad_sequences(tokenized, maxlen=max_len_answ, padding='post')
decoder_output = utils.to_categorical(padded, vocabulary_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import utils


In [ ]:
encoder_inputs = Input(shape=(max_len_quest))
encoder_embedding = Embedding(vocabulary_size, embedding_dim)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_len_answ))
decoder_embedding = Embedding(vocabulary_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocabulary_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()
# print(model.layers[-1].input)
print(model.layers[-3].output)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 62)]         0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 60, 256)      446208      ['input_7[0][0]']                
                                                                                                  
 embedding_6 (Embedding)        (None, 62, 256)      446208      ['input_8[0][0]']                
                                                                                            

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input, decoder_input], decoder_output,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [ ]:
tokenized = tokenizer.texts_to_sequences(['<eos> привет как жизнь'])
print(tokenized)
a = tokenizer.sequences_to_texts(tokenized)
print(a)

In [ ]:
def encode(target_seq, max_len):
  tokenized = tokenizer.texts_to_sequences(target_seq)
  padded = pad_sequences(tokenized, maxlen=max_len, padding='post')
  return np.array(padded)

In [ ]:
input_seq = ['<bos>']
input_seq = encode(input_seq, max_len_quest)
states_value = encoder_model.predict(input_seq)
target_seq = encode(['<bos>'], max_len_answ)
stop_condition = False
decoded_sentence = ''
while not stop_condition:
  output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
  sampled_token_index = np.argmax(output_tokens[0, -1, :])
  sampled_char = tokenizer.sequences_to_texts(str(sampled_token_index))
  decoded_sentence += str(sampled_char)
  if (sampled_char == '[START]' or len(decoded_sentence) > max_len_answ):
          stop_condition = True
  states_value = [h, c]
  target_seq = encode([sampled_char], max_len_answ)

In [ ]:
decoded_sentence